In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/yelp.csv


import nltk

nltk.download('wordnet')

In [2]:
data = pd.read_csv('/kaggle/input/yelp.csv')

In [3]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,26-01-2011,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,27-07-2011,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,14-06-2012,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,27-05-2010,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,05-01-2012,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
business_id    10000 non-null object
date           10000 non-null object
review_id      10000 non-null object
stars          10000 non-null int64
text           10000 non-null object
type           10000 non-null object
user_id        10000 non-null object
cool           10000 non-null int64
useful         10000 non-null int64
funny          10000 non-null int64
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [5]:
df = data[['text', 'stars']]

In [6]:
df = data[(data.stars==1)|(data.stars==5)]

df.text

In [7]:
x = df.text
y = df.stars

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=1)

In [10]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [11]:
lm = WordNetLemmatizer()

In [12]:
def lem(x):
    return lm.lemmatize(x)
xtrain = xtrain.apply(lem)
xtest = xtest.apply(lem)

In [13]:
import re

any([re.search('\w*[0-9]\w*', 'sdf sdfs 2323sd')])

a = 'hjghkj ewrwrwe 3wew4'
if any([re.search('\w*[0-9]\w*', a)]):
    print(re.search('\w*[0-9]\w*', a).group())

In [14]:
def rem(x):
    if (any([re.search('\w*[0-9]\w*', x)])):
#         print(x)
        try:
            sub = re.search('\w*[0-9]\w*', a).group()
        except:
            return x
        else:
            return x.replace(sub, '')
    else: return x

In [15]:
traindfx = xtrain.apply(rem)
testdfx = xtest.apply(rem)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [17]:
def vectoriser(ngram, min_df=1):
    global trainx, testx
    vect = CountVectorizer(stop_words='english', lowercase=True, ngram_range=ngram, min_df=min_df)
    trainx = vect.fit_transform(traindfx)
    print(trainx.shape)
    testx = vect.transform(testdfx)
#     print(vect.get_feature_names()[-100:])

In [18]:
ngram = (1,2)
mindf = 2
vectoriser(ngram, mindf)

(3064, 21629)


In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
model = MultinomialNB()
# model = SVC()
# model = LogisticRegression()

model.fit(trainx, ytrain)

print()
print()
print(model.score(testx, ytest))



0.9246575342465754


# Using Tensorflow

In [20]:
trainx

<3064x21629 sparse matrix of type '<class 'numpy.int64'>'
	with 178853 stored elements in Compressed Sparse Row format>

In [21]:
trainx.v

AttributeError: v not found

In [22]:
trainx = pd.DataFrame(trainx.toarray())
testx = pd.DataFrame(testx.toarray())
testy = ytest
trainy = ytrain

In [23]:
import tensorflow as tf

In [24]:
col = np.arange(0, 21629, 1)
col = [str(x) for x in col]

In [25]:
columns=[]
for i in col:
    columns.append(tf.feature_column.numeric_column(i))

In [26]:
est = tf.estimator.DNNClassifier([10, 10], columns)

In [27]:
def sinfn(batch_size=1):
    def infn():
        ipds = tf.data.Dataset.from_tensor_slices((dict(trainx), trainy))
        batchds = ipds.shuffle(1000).batch(batch_size)
        return batchds
    return infn

In [28]:
inputfn = sinfn()

In [29]:
est.train(input_fn=inputfn)

ValueError: in converted code:

    /opt/conda/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/canned/dnn.py:355 call  *
        net = self._input_layer(features)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:778 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/feature_column/dense_features.py:135 call
        self._state_manager)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:2852 get_dense_tensor
        return transformation_cache.get(self, state_manager)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:2615 get
        transformed = column.transform_feature(self, state_manager)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:2824 transform_feature
        input_tensor = transfor